# import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import keras_nlp
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

2024-08-11 14:35:51.290366: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 14:35:51.290496: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 14:35:51.423413: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Load data

In [3]:
# Load the datasets
train_data = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
test_data = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')

In [4]:
# Text Preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = tf.keras.preprocessing.text.text_to_word_sequence(text)
    return ' '.join(text)

train_data['full_text'] = train_data['full_text'].apply(preprocess_text)
test_data['full_text'] = test_data['full_text'].apply(preprocess_text)

# Encode the scores
train_labels = train_data['score'] - 1  

In [5]:
# Split the training data
X_train, X_val, y_train, y_val = train_test_split(train_data['full_text'], train_labels, test_size=0.2, random_state=42)

# Convert data to numpy arrays for the classifier
X_train = np.array(X_train.tolist())
X_val = np.array(X_val.tolist())
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

# model

In [6]:
# Pretrained classifier with keras_nlp
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en",
    num_classes=6,  


Attaching 'model.safetensors' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'task.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'model.safetensors' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'model.safetensors.index.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle notebook...
Attaching 'metadata.json' from model 'keras/bert/keras/bert_base_en/2' to your Kaggle 

In [7]:
# Compile the model
classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(5e-5),
    metrics=['accuracy'],
)

# Add early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_bert.keras', save_best_only=True, monitor='val_loss', mode='min')

In [8]:
# Train the model
history = classifier.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    epochs=5,  
    batch_size=8,  
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 1/5


I0000 00:00:1723388228.371447     114 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1723388228.450923     114 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1730/1731 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step - accuracy: 0.5034 - loss: 1.1819

W0000 00:00:1723388983.799786     114 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1731/1731 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step - accuracy: 0.5035 - loss: 1.1818

W0000 00:00:1723388989.690443     112 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1723389045.868402     114 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1731/1731 ━━━━━━━━━━━━━━━━━━━━ 945s 476ms/step - accuracy: 0.5035 - loss: 1.1818 - val_accuracy: 0.5586 - val_loss: 1.0154
Epoch 2/5
1731/1731 ━━━━━━━━━━━━━━━━━━━━ 712s 411ms/step - accuracy: 0.5882 - loss: 0.9565 - val_accuracy: 0.5254 - val_loss: 1.2356
Epoch 3/5
1731/1731 ━━━━━━━━━━━━━━━━━━━━ 718s 415ms/step - accuracy: 0.6027 - loss: 0.9477 - val_accuracy: 0.5690 - val_loss: 1.0126
Epoch 4/5
1731/1731 ━━━━━━━━━━━━━━━━━━━━ 716s 414ms/step - accuracy: 0.6425 - loss: 0.8551 - val_accuracy: 0.5765 - val_loss: 0.9953
Epoch 5/5
1731/1731 ━━━━━━━━━━━━━━━━━━━━ 710s 410ms/step - accuracy: 0.6307 - loss: 0.8588 - val_accuracy: 0.5725 - val_loss: 0.9964


# prediction on test data

In [9]:
# Load the best model
classifier.load_weights('best_model_bert.keras')

# Predict on the validation data
val_predictions = classifier.predict(x=X_val, batch_size=8).argmax(axis=-1) + 1  

# Calculate the quadratic weighted kappa score
kappa_score = cohen_kappa_score(y_val + 1, val_predictions, weights='quadratic') 
print(f'Validation Quadratic Weighted Kappa: {kappa_score}')

# Predict on the test data
test_predictions = classifier.predict(x=test_data['full_text'].tolist(), batch_size=8).argmax(axis=-1) + 1  

# Prepare the submission file
submission = pd.DataFrame({'essay_id': test_data['essay_id'], 'score': test_predictions})
submission.to_csv('submission.csv', index=False)
print("Submission file created successfully.")

  2/433 ━━━━━━━━━━━━━━━━━━━━ 56s 131ms/step

W0000 00:00:1723392271.655046     112 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


433/433 ━━━━━━━━━━━━━━━━━━━━ 60s 129ms/step


W0000 00:00:1723392327.301663     113 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Validation Quadratic Weighted Kappa: 0.7455186835768197
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Submission file created successfully.


W0000 00:00:1723392330.765440     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
